A (cluster model) + B (stabilizer features) + C (treatment outlier) Tambah D (Tweedie) dan E (hierarchy) sebagai refinement.

In [ ]:
import pandas as pd
import numpy as np

def build_sku_profile(df: pd.DataFrame) -> pd.DataFrame:
    df["qty"] = df["qty"].astype(float)

    profile = (
        df.groupby(["cabang", "sku"])
          .agg(
              n_months=("periode", "nunique"),
              qty_mean=("qty", "mean"),
              qty_std=("qty", "std"),
              qty_max=("qty", "max"),
              qty_min=("qty", "min"),
              total_qty=("qty", "sum"),
              zero_months=("qty", lambda x: (x == 0).sum()),
          )
          .reset_index()
    )

    profile["zero_ratio"] = profile["zero_months"] / profile["n_months"]
    profile["cv"] = profile["qty_std"] / profile["qty_mean"].replace(0, np.nan)

    # demand_level_bin
    profile["demand_level"] = pd.qcut(profile["qty_mean"], q=4, labels=[0,1,2,3])

    return profile


In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

def run_sku_clustering(profile: pd.DataFrame, n_clusters=4) -> pd.DataFrame:
    
    cluster_feats = ["qty_mean", "cv", "zero_ratio"]
    prof_clean = profile.dropna(subset=cluster_feats).copy()

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(prof_clean[cluster_feats].values)

    km = KMeans(n_clusters=n_clusters, random_state=1337, n_init="auto")
    prof_clean["cluster"] = km.fit_predict(X_scaled)

    profile = profile.merge(
        prof_clean[["cabang", "sku", "cluster"]],
        on=["cabang", "sku"],
        how="left"
    )

    profile["cluster"] = profile["cluster"].fillna(-1).astype(int)
    return profile


In [ ]:
import pandas as pd
import numpy as np

def add_stabilizer_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df_train = df[df["is_train"] == 1].copy()

    sku_stats = (
        df_train.groupby(["cabang","sku"])["qty"]
        .agg(["mean","std","count"])
        .reset_index()
    )
    sku_stats.columns = ["cabang","sku","qty_mean_cs","qty_std_cs","qty_cnt_cs"]
    df = df.merge(sku_stats, on=["cabang","sku"], how="left")
    df = df.sort_values(["cabang","sku","periode"])

    df["roll_mean_3"] = (
        df.groupby(["cabang","sku"])["qty"]
          .transform(lambda x: x.rolling(3, min_periods=1).mean())
    )
    df["roll_mean_12"] = (
        df.groupby(["cabang","sku"])["qty"]
          .transform(lambda x: x.rolling(12, min_periods=1).mean())
    )
    df["seasonal_ratio_12"] = df["roll_mean_3"] / (df["roll_mean_12"] + 1e-9)

    df["volatility_score"] = df["qty_std_cs"] / (df["qty_mean_cs"] + 1e-9)

    return df


In [ ]:
import pandas as pd

def winsorize_outliers(df: pd.DataFrame, clip_ratio=0.01) -> pd.DataFrame:
    df = df.copy()

    df_train = df[df["is_train"] == 1].copy()

    q_stats = (
        df_train.groupby(["cabang","sku"])["qty"]
        .quantile([clip_ratio, 1 - clip_ratio])
        .unstack()
        .reset_index()
    )
    q_stats.columns = ["cabang","sku","q_low","q_high"]

    df = df.merge(q_stats, on=["cabang","sku"], how="left")

    df["qty_wins"] = df["qty"]
    mask = df["q_low"].notna()

    df.loc[mask, "qty_wins"] = df.loc[mask].apply(
        lambda row: max(min(row["qty"], row["q_high"]), row["q_low"]), axis=1
    )

    return df


In [ ]:
import pandas as pd

def add_hierarchy_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    def sku_family(s):
        s = str(s).upper().strip()

        if s.endswith("CHAR"):
            return "char"
        if s.endswith("CPOX"):
            return "cpox"
        if s.endswith("CSW"):
            return "csw"
        if s.endswith("CSB") or "KSB" in s:
            return "csb"
        return s[:4]

    df["family"] = df["sku"].apply(sku_family)
    return df


In [ ]:
def get_tweedie_params():
    return {
        "objective": "tweedie",
        "tweedie_variance_power": 1.25,
        "metric": "rmse",
        "verbosity": -1,
        "force_row_wise": True,
        "seed": 1337,
    }


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import optuna
from optuna.samplers import TPESampler


def train_lgbm_per_cluster(
    df: pd.DataFrame,
    cluster_id: int,
    feature_cols: list,
    log_target=True,
    n_trials=40,
):
    df_c = df[df["cluster"] == cluster_id].copy()
    if df_c.empty:
        print("Cluster", cluster_id, "kosong. Skip.")
        return None

    df_c = df_c.sort_values(["cabang","sku","periode"]).reset_index(drop=True)

    if log_target:
        df_c["tgt"] = np.log1p(df_c["qty_wins"])
    else:
        df_c["tgt"] = df_c["qty_wins"]

    train_all = df_c[df_c["is_train"] == 1].copy()
    val_cutoff = pd.Timestamp("2024-02-01")

    train_inner = train_all[train_all["periode"] < val_cutoff]
    val_inner   = train_all[train_all["periode"] >= val_cutoff]

    if train_inner.empty or val_inner.empty:
        return None

    X_train = train_inner[feature_cols]
    X_val   = val_inner[feature_cols]

    y_train = train_inner["tgt"].values
    y_val   = val_inner["tgt"].values

    def objective(trial):
        params = get_tweedie_params()
        params.update({
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True),
            "num_leaves": trial.suggest_int("num_leaves", 31, 255),
            "max_depth": trial.suggest_int("max_depth", 3, 10),
            "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 200),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
            "bagging_freq": trial.suggest_int("bagging_freq", 1, 12),
        })

        train_set = lgb.Dataset(X_train, y_train)
        val_set   = lgb.Dataset(X_val, y_val)

        model = lgb.train(
            params,
            train_set,
            num_boost_round=2000,
            valid_sets=[train_set, val_set],
            valid_names=["train","val"],
            callbacks=[
                lgb.early_stopping(stopping_rounds=100, verbose=False)
            ],
        )

        pred_val = model.predict(X_val, num_iteration=model.best_iteration)

        if log_target:
            pred_val = np.expm1(pred_val)

        true_val = val_inner["qty"].values
        rmse_val = np.sqrt(np.mean((true_val - pred_val)**2))
        return rmse_val

    study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=1337))
    study.optimize(objective, n_trials=n_trials, show_progress_bar=False)

    best_iter = study.best_trial.user_attrs.get("best_iteration", 200)
    best_params = study.best_params

    final_params = get_tweedie_params()
    final_params.update(best_params)

    train_full = df_c[df_c["is_train"] == 1]
    X_full = train_full[feature_cols]
    y_full = train_full["tgt"].values

    model = lgb.train(
        final_params,
        lgb.Dataset(X_full, y_full),
        num_boost_round=best_iter,
    )

    return model


In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb

def load_cluster_models(path_dict):
    models = {}
    for cid, path in path_dict.items():
        models[cid] = lgb.Booster(model_file=str(path))
    return models

def predict_full(df, models, feature_cols):
    df = df.copy()
    preds = []

    for cid, model in models.items():
        df_c = df[df["cluster"] == cid].copy()
        if df_c.empty:
            continue

        pred = model.predict(df_c[feature_cols])
        pred = np.expm1(pred)
        df_c["pred_qty"] = pred
        preds.append(df_c)

    return pd.concat(preds, axis=0).sort_index()


In [ ]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from typing import Dict

import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

def mae(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return np.mean(np.abs(y_true - y_pred))


def mse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return np.mean((y_true - y_pred) ** 2)


def rmse(y_true, y_pred):
    return np.sqrt(mse(y_true, y_pred))


def mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(np.abs(y_true), eps)
    return np.mean(np.abs(y_true - y_pred) / denom) * 100.0


def smape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(np.abs(y_true) + np.abs(y_pred), eps)
    return np.mean(2.0 * np.abs(y_true - y_pred) / denom) * 100.0

PROJECT_ROOT   = Path(r"D:\Documents\Skripsi\demand-forecasting")
DATASET15_DIR  = PROJECT_ROOT / "data" / "dataset_15"

DATA_PATH      = DATASET15_DIR / "lgbm_dataset_15_fullfeat.csv"

OUT_ROOT       = PROJECT_ROOT / "outputs" / "lgbm_15_clusters_tweedie_noleak"
MODEL_DIR      = OUT_ROOT / "models"
METRIC_DIR     = OUT_ROOT / "metrics"
PLOT_DIR       = OUT_ROOT / "plots_per_series"
DIAG_DIR       = OUT_ROOT / "diagnostics"

for d in [OUT_ROOT, MODEL_DIR, METRIC_DIR, PLOT_DIR, DIAG_DIR]:
    d.mkdir(parents=True, exist_ok=True)


def main():
    print("====================================")
    print("RUN FULL TRAINING (NO LEAK): A+B+C+D+E")
    print("====================================")
    print("Load data:", DATA_PATH)

    df = pd.read_csv(DATA_PATH, parse_dates=["periode"])
    print("Rows:", len(df))

    df["qty"] = df["qty"].astype(float)
    df = df.sort_values(["cabang","sku","periode"]).reset_index(drop=True)

    print("\n[STEP 2] Build SKU profile dari TRAIN...")
    df_train = df[df["is_train"] == 1].copy()
    profile = build_sku_profile(df_train)
    PROFILE_PATH = DATASET15_DIR / "cluster_profiles_raw_train_only.csv"
    profile.to_csv(PROFILE_PATH, index=False)
    print("Saved raw train profile to:", PROFILE_PATH)

    print("\n[STEP 3] Clustering SKU (TRAIN only)...")
    profile_clustered = run_sku_clustering(profile, n_clusters=4)
    PROFILE_CLUSTER_PATH = DATASET15_DIR / "cluster_profiles_lgbm15_train_only.csv"
    profile_clustered.to_csv(PROFILE_CLUSTER_PATH, index=False)
    print("Saved clustered profile to:", PROFILE_CLUSTER_PATH)
    print("Cluster summary (train stats):")
    print(
        profile_clustered.groupby("cluster")[["qty_mean", "cv", "zero_ratio", "total_qty"]]
        .mean()
        .round(2)
        .to_string()
    )

    print("\n[STEP 4] Merge cluster dan demand_level ke panel (train+test)...")
    df = df.merge(
        profile_clustered[["cabang", "sku", "cluster", "demand_level"]],
        on=["cabang", "sku"],
        how="left",
    )
    df["cluster"] = df["cluster"].fillna(-1).astype(int)

    print("\n[STEP 5] Tambah hierarchy features (family)...")
    df = add_hierarchy_features(df)

    if "family" in df.columns:
        family_map = {
            fam: idx for idx, fam in enumerate(sorted(df["family"].astype(str).unique()))
        }
        df["family_idx"] = df["family"].astype(str).map(family_map).astype("int16")
        print("Family mapping:", family_map)

    print("\n[STEP 6] Tambah stabilizer features (no leak)...")
    df = add_stabilizer_features(df)

    print("\n[STEP 7] Winsorize outliers per SKU (no leak)...")
    df = winsorize_outliers(df)

    df["log_qty"] = np.log1p(df["qty"])
    df["log_qty_wins"] = np.log1p(df["qty_wins"])

    df = df.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)

    drop_cols = [
        "area",
        "cabang",
        "sku",
        "periode",
        "qty",
        "qty_wins",
        "log_qty",
        "log_qty_wins",
        "is_train",
        "is_test",
        "sample_weight",
        "family",
    ]

    feature_cols = [c for c in df.columns if c not in drop_cols]

    print("\n[STEP 8] Num features:", len(feature_cols))
    print("Contoh fitur:", feature_cols[:20])

    obj_cols = df[feature_cols].select_dtypes(include=["object"]).columns.tolist()
    if obj_cols:
        print("WARNING: Masih ada kolom object di feature_cols:", obj_cols)

    print("\n[STEP 9] Training LGBM per cluster (Tweedie, no leak)...")
    cluster_ids = sorted(df["cluster"].dropna().unique())
    models: Dict[int, lgb.Booster] = {}

    for cid in cluster_ids:
        if cid == -1:
            print(f"Cluster {cid} = -1 (unknown), skip training.")
            continue

        print("\n====================================")
        print(f"TRAINING CLUSTER {cid}")
        print("====================================")

        model = train_lgbm_per_cluster(
            df=df,
            cluster_id=int(cid),
            feature_cols=feature_cols,
            log_target=True,
            n_trials=40,
        )

        if model is None:
            print(f"Cluster {cid}: model is None, skip saving.")
            continue

        models[cid] = model

        model_path = MODEL_DIR / f"lgbm_15_cluster_{cid}.txt"
        model.save_model(str(model_path))
        print(f"Cluster {cid}: model saved to {model_path}")

    if not models:
        raise RuntimeError("Tidak ada model yang berhasil dilatih. Cek cluster atau flag is_train.")

    print("\n[STEP 10] Prediksi penuh (train + test) per cluster...")
    df_pred_list = []

    for cid, model in models.items():
        df_c = df[df["cluster"] == cid].copy()
        if df_c.empty:
            continue

        X_c = df_c[feature_cols]
        pred_log = model.predict(X_c)
        pred_qty = np.expm1(pred_log)

        df_c["pred_qty"] = pred_qty
        df_pred_list.append(df_c)

    df_pred = pd.concat(df_pred_list, axis=0).sort_index()
    PRED_PATH = OUT_ROOT / "panel_with_predictions.csv"
    df_pred.to_csv(PRED_PATH, index=False)
    print("Saved full panel with predictions to:", PRED_PATH)

    print("\n[STEP 11] Global metrics train/test...")
    metrics_global = []

    for split_name, mask in [
        ("train", df_pred["is_train"] == 1),
        ("test", df_pred["is_test"] == 1),
    ]:
        if not mask.any():
            continue

        yt = df_pred.loc[mask, "qty"].values
        yp = df_pred.loc[mask, "pred_qty"].values

        metrics_global.append({
            "split": split_name,
            "n_obs": int(len(yt)),
            "MSE": mse(yt, yp),
            "RMSE": rmse(yt, yp),
            "MAE": mae(yt, yp),
            "MAPE": mape(yt, yp),
            "sMAPE": smape(yt, yp),
        })

    global_df = pd.DataFrame(metrics_global)
    GLOBAL_METRIC_PATH = METRIC_DIR / "global_metrics_clusters_tweedie_noleak.csv"
    global_df.to_csv(GLOBAL_METRIC_PATH, index=False)
    print("Saved global metrics to:", GLOBAL_METRIC_PATH)
    print(global_df.to_string(index=False))

    print("\n[STEP 12] Metrics per cabang–SKU...")
    rows = []

    for (cab, sku), g in df_pred.groupby(["cabang", "sku"], sort=False):
        g_tr = g[g["is_train"] == 1]
        g_te = g[g["is_test"] == 1]

        row = {
            "cabang": cab,
            "sku": sku,
            "cluster": g["cluster"].iloc[0],
            "n_train": int(len(g_tr)),
            "n_test": int(len(g_te)),
        }

        if len(g_tr) > 0:
            yt_tr = g_tr["qty"].values
            yp_tr = g_tr["pred_qty"].values
            row.update({
                "train_mae": mae(yt_tr, yp_tr),
                "train_mse": mse(yt_tr, yp_tr),
                "train_rmse": rmse(yt_tr, yp_tr),
                "train_mape": mape(yt_tr, yp_tr),
                "train_smape": smape(yt_tr, yp_tr),
            })
        else:
            row.update({
                "train_mae": np.nan,
                "train_mse": np.nan,
                "train_rmse": np.nan,
                "train_mape": np.nan,
                "train_smape": np.nan,
            })

        if len(g_te) > 0:
            yt_te = g_te["qty"].values
            yp_te = g_te["pred_qty"].values
            row.update({
                "test_mae": mae(yt_te, yp_te),
                "test_mse": mse(yt_te, yp_te),
                "test_rmse": rmse(yt_te, yp_te),
                "test_mape": mape(yt_te, yp_te),
                "test_smape": smape(yt_te, yp_te),
            })
        else:
            row.update({
                "test_mae": np.nan,
                "test_mse": np.nan,
                "test_rmse": np.nan,
                "test_mape": np.nan,
                "test_smape": np.nan,
            })

        rows.append(row)

    metrics_series = pd.DataFrame(rows)
    metrics_series["gap_RMSE"] = metrics_series["test_rmse"] - metrics_series["train_rmse"]
    metrics_series["ratio_RMSE"] = metrics_series["test_rmse"] / metrics_series["train_rmse"]

    SERIES_METRIC_PATH = METRIC_DIR / "metrics_by_series_clusters_tweedie_noleak.csv"
    metrics_series.to_csv(SERIES_METRIC_PATH, index=False)
    print("Saved metrics per series to:", SERIES_METRIC_PATH)
    print(metrics_series.head(10).to_string(index=False))

    print(f"\n[STEP 13] Plot actual vs pred TEST per seri ke: {PLOT_DIR}")

    test_only = df_pred[df_pred["is_test"] == 1].copy()

    for (cab, sku), g in test_only.groupby(["cabang", "sku"], sort=False):
        g = g.sort_values("periode")

        if g["qty"].notna().sum() == 0:
            continue

        plt.figure(figsize=(10, 5))
        plt.plot(g["periode"], g["qty"], marker="o", label="Actual qty")
        plt.plot(g["periode"], g["pred_qty"], marker="x", label="Predicted qty")
        plt.xlabel("Periode")
        plt.ylabel("Qty")
        plt.title(f"Actual vs Predicted - TEST\nCabang {cab}, SKU {sku}")
        plt.legend()
        plt.grid(True)
        plt.tight_layout()

        fname = f"{cab}_{sku}_test_actual_vs_pred.png".replace("/", "-")
        plt.savefig(PLOT_DIR / fname, dpi=200)
        plt.close()

    print(f"\n[STEP 14] Diagnostics ke: {DIAG_DIR}")

    df_resid = df_pred.copy()
    df_resid["resid"] = df_resid["qty"].astype(float) - df_resid["pred_qty"].astype(float)
    df_resid["abs_resid"] = df_resid["resid"].abs()

    # Histogram residual global
    plt.figure(figsize=(8, 5))
    plt.hist(df_resid["resid"], bins=80)
    plt.xlabel("Residual (qty - pred_qty)")
    plt.ylabel("Frekuensi")
    plt.title("Histogram residual global (train + test)")
    plt.tight_layout()
    plt.savefig(DIAG_DIR / "hist_residual_global.png", dpi=200)
    plt.close()

    # Residual vs predicted
    plt.figure(figsize=(8, 5))
    plt.scatter(df_resid["pred_qty"], df_resid["resid"], alpha=0.3)
    plt.axhline(0, color="red", linestyle="--")
    plt.xlabel("Predicted qty")
    plt.ylabel("Residual")
    plt.title("Residual vs predicted qty")
    plt.tight_layout()
    plt.savefig(DIAG_DIR / "scatter_resid_vs_pred.png", dpi=200)
    plt.close()

    # Top outliers (train)
    TOP_N = 50
    top_outliers = (
        df_resid[df_resid["is_train"] == 1]
        .sort_values("abs_resid", ascending=False)
        .head(TOP_N)
        [["area", "cabang", "sku", "periode", "qty", "pred_qty", "resid", "abs_resid"]]
    )
    OUTLIER_PATH = DIAG_DIR / "top_outliers_train.csv"
    top_outliers.to_csv(OUTLIER_PATH, index=False)
    print("Saved top outliers train to:", OUTLIER_PATH)

    # Hist ratio_RMSE
    plt.figure(figsize=(8, 5))
    plt.hist(metrics_series["ratio_RMSE"].dropna(), bins=30)
    plt.xlabel("ratio_RMSE = test_rmse / train_rmse")
    plt.ylabel("Jumlah seri")
    plt.title("Distribusi ratio_RMSE antar seri")
    plt.tight_layout()
    plt.savefig(DIAG_DIR / "hist_ratio_RMSE.png", dpi=200)
    plt.close()

    # Scatter train vs test RMSE
    plt.figure(figsize=(6, 6))
    plt.scatter(metrics_series["train_rmse"], metrics_series["test_rmse"], alpha=0.7)
    max_val = np.nanmax([
        metrics_series["train_rmse"].max(),
        metrics_series["test_rmse"].max()
    ])
    plt.plot([0, max_val], [0, max_val], "r--")
    plt.xlabel("Train RMSE")
    plt.ylabel("Test RMSE")
    plt.title("Train vs Test RMSE per cabang–SKU")
    plt.tight_layout()
    plt.savefig(DIAG_DIR / "scatter_train_vs_test_RMSE.png", dpi=200)
    plt.close()

    # Seri overfit / underfit
    overfit_series = metrics_series[metrics_series["ratio_RMSE"] > 1.3].copy()
    under_series   = metrics_series[metrics_series["ratio_RMSE"] < 0.8].copy()

    print("\nSeri dengan ratio_RMSE > 1.3 (indikasi sulit di test / overfit lokal):")
    if len(overfit_series) > 0:
        print(
            overfit_series[["cabang", "sku", "cluster", "train_rmse", "test_rmse", "ratio_RMSE"]]
            .sort_values("ratio_RMSE", ascending=False)
            .head(20)
            .to_string(index=False)
        )
    else:
        print("Tidak ada.")

    print("\nSeri dengan ratio_RMSE < 0.8 (train lebih jelek dari test):")
    if len(under_series) > 0:
        print(
            under_series[["cabang", "sku", "cluster", "train_rmse", "test_rmse", "ratio_RMSE"]]
            .sort_values("ratio_RMSE")
            .head(20)
            .to_string(index=False)
        )
    else:
        print("Tidak ada.")

    print("\nSELESAI: A+B+C+D+E (NO LEAK) + diagnostics lengkap.")


if __name__ == "__main__":
    main()


RUN FULL TRAINING (NO LEAK): A+B+C+D+E
Load data: D:\Documents\Skripsi\demand-forecasting\data\dataset_15\lgbm_dataset_15_fullfeat.csv
Rows: 4965

[STEP 2] Build SKU profile dari TRAIN...
Saved raw train profile to: D:\Documents\Skripsi\demand-forecasting\data\dataset_15\cluster_profiles_raw_train_only.csv

[STEP 3] Clustering SKU (TRAIN only)...
Saved clustered profile to: D:\Documents\Skripsi\demand-forecasting\data\dataset_15\cluster_profiles_lgbm15_train_only.csv
Cluster summary (train stats):
         qty_mean    cv  zero_ratio  total_qty
cluster                                       
0         1382.52  0.39         0.0   56683.43
1         4741.34  0.54         0.0  194394.92
2         3926.01  1.14         0.0  160966.57
3         1623.43  0.80         0.0   66560.77

[STEP 4] Merge cluster dan demand_level ke panel (train+test)...

[STEP 5] Tambah hierarchy features (family)...
Family mapping: {'APQR': 0, 'ATUV': 1, 'AUVW': 2, 'BBCD': 3, 'BKLM': 4, 'BUVW': 5, 'BVWX': 6, 'CFGH':

[I 2025-11-23 20:51:22,806] A new study created in memory with name: no-name-a403765d-c47e-4402-b153-9cc84b24759a



[STEP 8] Num features: 40
Contoh fitur: ['event_flag', 'event_flag_lag1', 'holiday_count', 'holiday_count_lag1', 'rainfall_lag1', 'imputed', 'spike_flag', 'month', 'year', 'qtr', 'qty_lag1', 'qty_lag2', 'qty_lag3', 'qty_lag4', 'qty_lag5', 'qty_lag6', 'qty_lag7', 'qty_lag8', 'qty_lag9', 'qty_lag10']

[STEP 9] Training LGBM per cluster (Tweedie, no leak)...

TRAINING CLUSTER 0


[I 2025-11-23 20:51:26,936] Trial 0 finished with value: 121.59964812496591 and parameters: {'learning_rate': 0.021923099845827587, 'num_leaves': 66, 'max_depth': 5, 'min_data_in_leaf': 103, 'feature_fraction': 0.7284002162080668, 'bagging_fraction': 0.8073571282390148, 'bagging_freq': 4}. Best is trial 0 with value: 121.59964812496591.
[I 2025-11-23 20:51:28,538] Trial 1 finished with value: 126.00291693328303 and parameters: {'learning_rate': 0.18617280148093396, 'num_leaves': 195, 'max_depth': 3, 'min_data_in_leaf': 89, 'feature_fraction': 0.8514004718158846, 'bagging_fraction': 0.6500231705342397, 'bagging_freq': 12}. Best is trial 0 with value: 121.59964812496591.
[I 2025-11-23 20:51:33,919] Trial 2 finished with value: 107.21132632409115 and parameters: {'learning_rate': 0.03772670263489984, 'num_leaves': 208, 'max_depth': 9, 'min_data_in_leaf': 85, 'feature_fraction': 0.7664415758283992, 'bagging_fraction': 0.8337032512441286, 'bagging_freq': 10}. Best is trial 2 with value: 107

Cluster 0: model saved to D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_clusters_tweedie_noleak\models\lgbm_15_cluster_0.txt

TRAINING CLUSTER 1


...

Cluster 1: model saved to D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_clusters_tweedie_noleak\models\lgbm_15_cluster_1.txt

TRAINING CLUSTER 2


[I 2025-11-23 20:55:34,220] Trial 0 finished with value: 5019.127933989106 and parameters: {'learning_rate': 0.021923099845827587, 'num_leaves': 66, 'max_depth': 5, 'min_data_in_leaf': 103, 'feature_fraction': 0.7284002162080668, 'bagging_fraction': 0.8073571282390148, 'bagging_freq': 4}. Best is trial 0 with value: 5019.127933989106.
[I 2025-11-23 20:55:34,331] Trial 1 finished with value: 4483.913725156995 and parameters: {'learning_rate': 0.18617280148093396, 'num_leaves': 195, 'max_depth': 3, 'min_data_in_leaf': 89, 'feature_fraction': 0.8514004718158846, 'bagging_fraction': 0.6500231705342397, 'bagging_freq': 12}. Best is trial 1 with value: 4483.913725156995.
[I 2025-11-23 20:55:34,540] Trial 2 finished with value: 4594.299331087771 and parameters: {'learning_rate': 0.03772670263489984, 'num_leaves': 208, 'max_depth': 9, 'min_data_in_leaf': 85, 'feature_fraction': 0.7664415758283992, 'bagging_fraction': 0.8337032512441286, 'bagging_freq': 10}. Best is trial 1 with value: 4483.913

Cluster 2: model saved to D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_clusters_tweedie_noleak\models\lgbm_15_cluster_2.txt

TRAINING CLUSTER 3


[I 2025-11-23 20:55:45,686] Trial 0 finished with value: 613.9524689397414 and parameters: {'learning_rate': 0.021923099845827587, 'num_leaves': 66, 'max_depth': 5, 'min_data_in_leaf': 103, 'feature_fraction': 0.7284002162080668, 'bagging_fraction': 0.8073571282390148, 'bagging_freq': 4}. Best is trial 0 with value: 613.9524689397414.
[I 2025-11-23 20:55:45,925] Trial 1 finished with value: 657.0625738189198 and parameters: {'learning_rate': 0.18617280148093396, 'num_leaves': 195, 'max_depth': 3, 'min_data_in_leaf': 89, 'feature_fraction': 0.8514004718158846, 'bagging_fraction': 0.6500231705342397, 'bagging_freq': 12}. Best is trial 0 with value: 613.9524689397414.
[I 2025-11-23 20:55:47,039] Trial 2 finished with value: 537.1565237500789 and parameters: {'learning_rate': 0.03772670263489984, 'num_leaves': 208, 'max_depth': 9, 'min_data_in_leaf': 85, 'feature_fraction': 0.7664415758283992, 'bagging_fraction': 0.8337032512441286, 'bagging_freq': 10}. Best is trial 2 with value: 537.1565

Cluster 3: model saved to D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_clusters_tweedie_noleak\models\lgbm_15_cluster_3.txt

[STEP 10] Prediksi penuh (train + test) per cluster...
Saved full panel with predictions to: D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_clusters_tweedie_noleak\panel_with_predictions.csv

[STEP 11] Global metrics train/test...
Saved global metrics to: D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_clusters_tweedie_noleak\metrics\global_metrics_clusters_tweedie_noleak.csv
split  n_obs           MSE       RMSE        MAE      MAPE     sMAPE
train   4920 313683.507019 560.074555 173.828192  8.962125  8.243627
 test     45 626566.313539 791.559419 512.546156 16.429938 15.987120

[STEP 12] Metrics per cabang–SKU...
Saved metrics per series to: D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_clusters_tweedie_noleak\metrics\metrics_by_series_clusters_tweedie_noleak.csv
cabang          sku  cluster  n_train  n_test  train_mae     

Ini cek semua terhadap data eksternal berpengaruh ga 

In [ ]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
from typing import Dict, List, Optional
import itertools
import inspect

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def mse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    return float(np.mean((y_true - y_pred) ** 2))

def rmse(y_true, y_pred):
    return float(np.sqrt(mse(y_true, y_pred)))

def mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    denom = np.maximum(np.abs(y_true), eps)
    return float(np.mean(np.abs(y_true - y_pred) / denom) * 100.0)

from app.profiling.sku_profiler import build_sku_profile
from app.profiling.clustering import run_sku_clustering
from app.features.hierarchy_features import add_hierarchy_features
from app.features.stabilizer_features import add_stabilizer_features
from app.features.outlier_handler import winsorize_outliers
from app.modeling.lgbm_trainer_cluster import train_lgbm_per_cluster

def _ensure_int_flags(df: pd.DataFrame) -> pd.DataFrame:
    for c in ["is_train", "is_test"]:
        if c in df.columns:
            df[c] = df[c].astype(int)
    return df

def _safe_feature_cols(df: pd.DataFrame, cols: List[str]) -> List[str]:
    cols = [c for c in cols if c in df.columns]
    obj_cols = df[cols].select_dtypes(include=["object"]).columns.tolist()
    if obj_cols:
        raise ValueError(f"Masih ada kolom object di feature_cols: {obj_cols}")
    return cols

def build_sku_profile_adapter(df_train: pd.DataFrame) -> pd.DataFrame:
    sig = inspect.signature(build_sku_profile)
    params = list(sig.parameters.keys())

    if len(params) == 1:
        return build_sku_profile(df_train)

    train_end = str(pd.to_datetime(df_train["periode"]).max().date())
    if "train_end" in params:
        return build_sku_profile(df_train, train_end=train_end)

    return build_sku_profile(df_train, train_end)

def _label_ext(cols: List[str]) -> str:
    if not cols:
        return "Tanpa Eksternal"
    pretty = {
        "event_flag": "Event",
        "event_flag_lag1": "Event (Lag-1)",
        "holiday_count": "Libur",
        "holiday_count_lag1": "Libur (Lag-1)",
        "rainfall_lag1": "Hujan (Lag-1)",
    }
    order = ["event_flag", "event_flag_lag1", "holiday_count", "holiday_count_lag1", "rainfall_lag1"]
    cols_sorted = [c for c in order if c in cols]
    return " + ".join([pretty.get(c, c) for c in cols_sorted])

PROJECT_ROOT = Path(r"D:\Documents\Skripsi\demand-forecasting")
DATA_PATH = PROJECT_ROOT / "data" / "dataset_15" / "lgbm_dataset_15_fullfeat.csv"

OUT_ROOT = PROJECT_ROOT / "outputs" / "lgbm_15_ablation_external_follow_baseline"
PLOT_ENABLE = False  

def main():
    OUT_ROOT.mkdir(parents=True, exist_ok=True)
    if PLOT_ENABLE:
        (OUT_ROOT / "plots").mkdir(parents=True, exist_ok=True)

    print("Load:", DATA_PATH)
    df = pd.read_csv(DATA_PATH, parse_dates=["periode"])
    df["qty"] = df["qty"].astype(float)
    df = _ensure_int_flags(df)
    df = df.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)
    print("Rows:", len(df))

    df_train = df[df["is_train"] == 1].copy()
    profile = build_sku_profile_adapter(df_train)
    profile_clustered = run_sku_clustering(profile, n_clusters=4)

    df = df.merge(
        profile_clustered[["cabang", "sku", "cluster", "demand_level"]],
        on=["cabang", "sku"],
        how="left",
    )
    df["cluster"] = df["cluster"].fillna(-1).astype(int)

    df = add_hierarchy_features(df)

    if "family" in df.columns:
        family_map = {fam: idx for idx, fam in enumerate(sorted(df["family"].astype(str).unique()))}
        df["family_idx"] = df["family"].astype(str).map(family_map).astype("int16")

    df = add_stabilizer_features(df)
    df = winsorize_outliers(df)

    df["log_qty"] = np.log1p(df["qty"])
    df["log_qty_wins"] = np.log1p(df["qty_wins"])
    df = df.sort_values(["cabang", "sku", "periode"]).reset_index(drop=True)

    drop_cols = [
        "area",
        "cabang",
        "sku",
        "periode",
        "qty",
        "qty_wins",
        "log_qty",
        "log_qty_wins",
        "is_train",
        "is_test",
        "sample_weight",  
        "family",
    ]
    drop_cols = [c for c in drop_cols if c in df.columns]

    feature_cols_baseline = [c for c in df.columns if c not in drop_cols]
    feature_cols_baseline = _safe_feature_cols(df, feature_cols_baseline)
    external_all = [
        "event_flag",
        "event_flag_lag1",
        "holiday_count",
        "holiday_count_lag1",
        "rainfall_lag1",
    ]
    external_all = [c for c in external_all if c in df.columns]
    external_all = [c for c in external_all if c in feature_cols_baseline]

    if not external_all:
        raise RuntimeError("Tidak ada kolom eksternal yang ditemukan di feature pool baseline.")

    base_non_external = [c for c in feature_cols_baseline if c not in external_all]
    base_non_external = _safe_feature_cols(df, base_non_external)

    print("Baseline feature count:", len(feature_cols_baseline))
    print("Non-external base count:", len(base_non_external))
    print("External cols used for ablation:", external_all)
    combos: List[List[str]] = []
    for r in range(0, len(external_all) + 1):
        for combo in itertools.combinations(external_all, r):
            combos.append(list(combo))
    combos = sorted(combos, key=lambda x: (len(x), x))

    cfg_map_rows = []
    for i, ext_cols in enumerate(combos):
        cfg_id = f"CFG_{i:02d}"
        cfg_map_rows.append({
            "config_id": cfg_id,
            "data_eksternal": _label_ext(ext_cols),
            "external_used": ", ".join(ext_cols) if ext_cols else "-",
        })
    pd.DataFrame(cfg_map_rows).to_csv(OUT_ROOT / "CONFIG_MAPPING.csv", index=False)
    print("Total configs:", len(combos))
    print("Saved mapping:", OUT_ROOT / "CONFIG_MAPPING.csv")
    out_rows = []

    cluster_ids = sorted(df["cluster"].dropna().unique())
    cluster_ids = [int(c) for c in cluster_ids if int(c) != -1]

    for i, ext_cols in enumerate(combos):
        cfg_id = f"CFG_{i:02d}"
        cfg_label = _label_ext(ext_cols)
        ext_cols = [c for c in ext_cols if c in df.columns] 

        feature_cols = _safe_feature_cols(df, base_non_external + ext_cols)

        print("\n===============================")
        print("CONFIG:", cfg_id, "|", cfg_label)
        print("External:", ext_cols if ext_cols else "-")
        print("Feature count:", len(feature_cols))
        print("===============================")

        models = {}
        for cid in cluster_ids:
            model = train_lgbm_per_cluster(
                df=df,
                cluster_id=int(cid),
                feature_cols=feature_cols,
                log_target=True,
                n_trials=40,
            )
            if model is None:
                continue
            models[int(cid)] = model

        if not models:
            print("No models trained. Skip.")
            continue

        df_pred_parts = []
        for cid, model in models.items():
            df_c = df[df["cluster"] == cid].copy()
            if df_c.empty:
                continue

            pred_log = model.predict(df_c[feature_cols])
            df_c["pred_qty"] = np.expm1(pred_log)
            df_pred_parts.append(df_c)

        if not df_pred_parts:
            print("No predictions. Skip.")
            continue

        df_pred = pd.concat(df_pred_parts, axis=0).sort_values(["cabang","sku","periode"]).reset_index(drop=True)

        test_only = df_pred[df_pred["is_test"] == 1].copy()
        for (cab, sku), g in test_only.groupby(["cabang", "sku"], sort=False):
            yt = g["qty"].astype(float).values
            yp = g["pred_qty"].astype(float).values
            out_rows.append({
                "cabang": cab,
                "sku": sku,
                "config_id": cfg_id,
                "data_eksternal": cfg_label,
                "external_used": ", ".join(ext_cols) if ext_cols else "-",
                "rmse_test": rmse(yt, yp),
                "mape_test": mape(yt, yp),
            })

            if PLOT_ENABLE:
                plt.figure(figsize=(9, 4.5))
                gg = g.sort_values("periode")
                plt.plot(gg["periode"], gg["qty"], marker="o", label="Actual")
                plt.plot(gg["periode"], gg["pred_qty"], marker="x", label="Pred")
                plt.title(f"{cfg_id} | {cab}-{sku} | {cfg_label}")
                plt.xlabel("Periode")
                plt.ylabel("Qty")
                plt.grid(True)
                plt.legend()
                plt.tight_layout()
                fname = f"{cfg_id}_{cab}_{sku}.png".replace("/", "-")
                plt.savefig(OUT_ROOT / "plots" / fname, dpi=160)
                plt.close()

        print("Done:", cfg_id)

    if not out_rows:
        raise RuntimeError("Tidak ada output. Cek is_test di dataset.")

    df_out = pd.DataFrame(out_rows)
    df_out = df_out.sort_values(["cabang","sku","config_id"]).reset_index(drop=True)

    out_path = OUT_ROOT / "METRICS_LONG_TEST_only.csv"
    df_out.to_csv(out_path, index=False)

    print("\nSAVED:", out_path)
    print("Rows:", len(df_out))
    print("SELESAI.")

if __name__ == "__main__":
    main()


Load: D:\Documents\Skripsi\demand-forecasting\data\dataset_15\lgbm_dataset_15_fullfeat.csv
Rows: 4965


[I 2026-01-02 18:27:22,932] A new study created in memory with name: no-name-1ef2106b-28fd-4fc9-b16d-d9079c4cf072


Baseline feature count: 40
Non-external base count: 35
External cols used for ablation: ['event_flag', 'event_flag_lag1', 'holiday_count', 'holiday_count_lag1', 'rainfall_lag1']
Total configs: 32
Saved mapping: D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_ablation_external_follow_baseline\CONFIG_MAPPING.csv

CONFIG: CFG_00 | Tanpa Eksternal
External: -
Feature count: 35


[I 2026-01-02 18:27:24,464] Trial 0 finished with value: 100.84521563990185 and parameters: {'learning_rate': 0.021923099845827587, 'num_leaves': 66, 'max_depth': 5, 'min_data_in_leaf': 103, 'feature_fraction': 0.7284002162080668, 'bagging_fraction': 0.8073571282390148, 'bagging_freq': 4}. Best is trial 0 with value: 100.84521563990185.
[I 2026-01-02 18:27:24,847] Trial 1 finished with value: 124.63497275890755 and parameters: {'learning_rate': 0.18617280148093396, 'num_leaves': 195, 'max_depth': 3, 'min_data_in_leaf': 89, 'feature_fraction': 0.8514004718158846, 'bagging_fraction': 0.6500231705342397, 'bagging_freq': 12}. Best is trial 0 with value: 100.84521563990185.
[I 2026-01-02 18:27:26,545] Trial 2 finished with value: 96.12540768091485 and parameters: {'learning_rate': 0.03772670263489984, 'num_leaves': 208, 'max_depth': 9, 'min_data_in_leaf': 85, 'feature_fraction': 0.7664415758283992, 'bagging_fraction': 0.8337032512441286, 'bagging_freq': 10}. Best is trial 2 with value: 96.1

Done: CFG_00

CONFIG: CFG_01 | Event
External: ['event_flag']
Feature count: 36


...

Done: CFG_29

CONFIG: CFG_30 | Event (Lag-1) + Libur + Libur (Lag-1) + Hujan (Lag-1)
External: ['event_flag_lag1', 'holiday_count', 'holiday_count_lag1', 'rainfall_lag1']
Feature count: 39


[I 2026-01-02 19:17:36,999] Trial 0 finished with value: 114.72663209231908 and parameters: {'learning_rate': 0.021923099845827587, 'num_leaves': 66, 'max_depth': 5, 'min_data_in_leaf': 103, 'feature_fraction': 0.7284002162080668, 'bagging_fraction': 0.8073571282390148, 'bagging_freq': 4}. Best is trial 0 with value: 114.72663209231908.
[I 2026-01-02 19:17:37,527] Trial 1 finished with value: 135.77235462009884 and parameters: {'learning_rate': 0.18617280148093396, 'num_leaves': 195, 'max_depth': 3, 'min_data_in_leaf': 89, 'feature_fraction': 0.8514004718158846, 'bagging_fraction': 0.6500231705342397, 'bagging_freq': 12}. Best is trial 0 with value: 114.72663209231908.
[I 2026-01-02 19:17:39,461] Trial 2 finished with value: 97.32340919058348 and parameters: {'learning_rate': 0.03772670263489984, 'num_leaves': 208, 'max_depth': 9, 'min_data_in_leaf': 85, 'feature_fraction': 0.7664415758283992, 'bagging_fraction': 0.8337032512441286, 'bagging_freq': 10}. Best is trial 2 with value: 97.3

Done: CFG_30

CONFIG: CFG_31 | Event + Event (Lag-1) + Libur + Libur (Lag-1) + Hujan (Lag-1)
External: ['event_flag', 'event_flag_lag1', 'holiday_count', 'holiday_count_lag1', 'rainfall_lag1']
Feature count: 40


[I 2026-01-02 19:19:19,867] Trial 0 finished with value: 111.83323968138015 and parameters: {'learning_rate': 0.021923099845827587, 'num_leaves': 66, 'max_depth': 5, 'min_data_in_leaf': 103, 'feature_fraction': 0.7284002162080668, 'bagging_fraction': 0.8073571282390148, 'bagging_freq': 4}. Best is trial 0 with value: 111.83323968138015.
[I 2026-01-02 19:19:20,271] Trial 1 finished with value: 117.81289034312849 and parameters: {'learning_rate': 0.18617280148093396, 'num_leaves': 195, 'max_depth': 3, 'min_data_in_leaf': 89, 'feature_fraction': 0.8514004718158846, 'bagging_fraction': 0.6500231705342397, 'bagging_freq': 12}. Best is trial 0 with value: 111.83323968138015.
[I 2026-01-02 19:19:22,252] Trial 2 finished with value: 102.87743501728586 and parameters: {'learning_rate': 0.03772670263489984, 'num_leaves': 208, 'max_depth': 9, 'min_data_in_leaf': 85, 'feature_fraction': 0.7664415758283992, 'bagging_fraction': 0.8337032512441286, 'bagging_freq': 10}. Best is trial 2 with value: 102

Done: CFG_31

SAVED: D:\Documents\Skripsi\demand-forecasting\outputs\lgbm_15_ablation_external_follow_baseline\METRICS_LONG_TEST_only.csv
Rows: 288
SELESAI.
